<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 🔔 課表查詢 + 上課提醒 + AI 行前準備（Gradio + Colab 版）
# 使用方式：
# 1. 在 Colab 新增一個程式碼格，貼上這整段程式
# 2. 先執行一次
# 3. 下面會出現一個網頁介面（Gradio），直接在上面操作即可

!pip install gradio -q

import pandas as pd
from datetime import datetime, timedelta, time
import gradio as gr

# === 1. 讀取 Google 試算表 ===
# 請確認該試算表「任何知道連結的人都可以檢視」
# 你的表：41371113H_彭奕憲_課表
sheet_id = "1Qx3lwHyaJfo48elzq0mspGxNnamn1FMtOIldYk8oi-8"
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

# 預期欄位：
# 學號, 姓名, 系所, 班級, 星期, 節次, 上課時間, 課程中文名稱, 上課教室

def parse_start_time(time_str):
    """
    將『14:20 - 15:10』這種字串，取出開始時間 14:20 轉成 datetime.time
    若格式不符則回傳 None（避免整個程式掛掉）
    """
    if not isinstance(time_str, str):
        return None
    try:
        start_part = time_str.split("-")[0].strip()  # "14:20"
        hour, minute = start_part.split(":")
        return time(int(hour), int(minute))
    except Exception:
        return None

# 讀取一次課表資料
df = pd.read_csv(csv_url)
df["開始時間_obj"] = df["上課時間"].apply(parse_start_time)

# === 2. 星期對照表 ===

weekday_map_num_to_tw = {
    0: "星期一",
    1: "星期二",
    2: "星期三",
    3: "星期四",
    4: "星期五",
    5: "星期六",
    6: "星期日",
}

# === 3. 查詢某一天的課表（給 Gradio 用） ===

def query_by_weekday_gr(weekday_tw):
    """
    Gradio 呼叫用：輸入『星期一』這種字串，輸出排版好的文字
    """
    if weekday_tw is None or weekday_tw == "":
        return "⚠️ 請先選擇一個星期！"

    mask = df["星期"] == weekday_tw
    day_df = df[mask].sort_values(by="節次")

    if day_df.empty:
        return f"📅 {weekday_tw} 沒有課喔～"

    lines = [f"📅 {weekday_tw} 課表：", ""]
    for _, row in day_df.iterrows():
        lines.append(
            f"節次：{row['節次']}\n"
            f"時間：{row['上課時間']}\n"
            f"課程：{row['課程中文名稱']}\n"
            f"地點：{row['上課教室']}\n"
            "--------------------------"
        )
    return "\n".join(lines)

# === 4. 今天／一段時間內的上課提醒（給 Gradio 用） ===

def upcoming_classes_gr(minutes_ahead):
    """
    Gradio 呼叫用：
    查詢『今天接下來 N 分鐘內』即將開始的課程
    """
    try:
        minutes_ahead = int(minutes_ahead)
    except:
        return "⚠️ 請輸入有效的分鐘數（正整數）"

    if minutes_ahead <= 0:
        return "⚠️ 分鐘數需大於 0"

    now = datetime.now()
    today_py_weekday = now.weekday()  # 0~6
    today_tw = weekday_map_num_to_tw.get(today_py_weekday, "未知星期")

    now_time = now.time()
    later_time = (now + timedelta(minutes=minutes_ahead)).time()

    today_df = df[df["星期"] == today_tw].copy()
    today_df = today_df.dropna(subset=["開始時間_obj"])

    # 過濾：開始時間在 [現在, 現在+N 分鐘] 之間
    upcoming = today_df[
        (today_df["開始時間_obj"] >= now_time) &
        (today_df["開始時間_obj"] <= later_time)
    ].sort_values(by="開始時間_obj")

    header_lines = [
        f"⏰ 現在時間：{now.strftime('%Y-%m-%d %H:%M')}",
        f"📅 今天是：{today_tw}",
        "",
    ]

    if upcoming.empty:
        header_lines.append(f"未來 {minutes_ahead} 分鐘內沒有新的課要上～")
        return "\n".join(header_lines)

    header_lines.append(f"未來 {minutes_ahead} 分鐘內即將開始的課程：")
    header_lines.append("")

    for _, row in upcoming.iterrows():
        header_lines.append(
            f"時間：{row['上課時間']}\n"
            f"課程：{row['課程中文名稱']}\n"
            f"地點：{row['上課教室']}\n"
            "--------------------------"
        )

    return "\n".join(header_lines)

# === 5. AI 行前準備建議（rule-based 小AI） ===

def ai_prep_tips(weekday_tw):
    """
    根據某一天的課程，給出行前準備建議（純本地規則，不需外部 API）
    """
    if weekday_tw is None or weekday_tw == "":
        return "⚠️ 請先選擇一個星期！"

    mask = df["星期"] == weekday_tw
    day_df = df[mask].sort_values(by="節次")

    if day_df.empty:
        return f"📅 {weekday_tw} 沒有課，不用特別準備～好好休息 😆"

    lines = [f"🤖 {weekday_tw} 行前準備建議", ""]
    lines.append("📚 今天你有這些課：")
    for _, row in day_df.iterrows():
        lines.append(f"- {row['上課時間']}《{row['課程中文名稱']}》 @ {row['上課教室']}")
    lines.append("")

    # 整體通用建議
    lines.append("🧠 全日通用建議：")
    lines.append("• 基本配備：學生證、錢包、悠遊卡/一卡通。")
    lines.append("• 文具：筆記本、原子筆/自動鉛筆、橡皮擦。")
    lines.append("• 3C：充飽電的手機，建議帶行動電源。")
    lines.append("• 其他：隨身水壺，留意天氣帶雨具或外套。")
    lines.append("")

    # 依課程類型給更細的建議
    lines.append("🎯 依課程的加強提醒：")

    for _, row in day_df.iterrows():
        cname = str(row["課程中文名稱"])
        ctime = row["上課時間"]
        room = row["上課教室"]

        tips = []

        # 體育 / 運動類
        if any(k in cname for k in ["體育", "運動", "Sport", "sports"]):
            tips.append("運動服、運動鞋、毛巾、水壺，避免配戴貴重飾品。")

        # 實驗 / 實習 / 實作 / lab
        if any(k in cname for k in ["實驗", "實習", "實作", "Lab", "LAB", "lab"]):
            tips.append("實驗衣（如有要求）、完成的預報/講義、USB 或雲端備份資料。")

        # 英文 / English 類
        if any(k in cname for k in ["英文", "English", "英語"]):
            tips.append("英文課本、單字本、前一週的上課講義，若有聽力課可準備有線耳機。")

        # 資訊 / 電腦 / 資工 類
        if any(k in cname for k in ["資訊", "程式", "Programming", "電腦", "資訊概論"]):
            tips.append("筆電（若需要）、充電器、隨身碟，並確認相關帳號/系統可以登入。")

        # 通識 / 報告類
        if any(k in cname for k in ["通識", "概論", "管理", "報告"]):
            tips.append("已完成的作業或報告進度，預先想好可在課堂發問或討論的題目。")

        # 若以上都沒有命中，就給通用學科建議
        if not tips:
            tips.append("課本、講義、筆記本，提前 5 分鐘到教室整理心情。")

        lines.append(f"《{cname}》 ({ctime} @ {room})")
        for t in tips:
            lines.append(f"• 建議準備：{t}")
        lines.append("")

    # 小收尾
    lines.append("✅ 小提醒：前一天晚上先把包包收好，早上出門前只要檢查一次就好，減少遺漏。")
    lines.append("✅ 若有作業或小考，建議出門前再快速確認一次是否帶齊。")

    return "\n".join(lines)

# === 6. Gradio 介面 ===

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 🗓 課表查詢 & 上課提醒 & 🤖 行前準備小助理
        - Tab1：選星期 → 查當天課表
        - Tab2：設定 N 分鐘 → 看今天接下來要上的課
        - Tab3：AI 行前準備 → 告訴你今天大概要帶什麼、注意什麼
        """
    )

    # Tab 1：依星期查詢課表
    with gr.Tab("📅 依星期查詢課表"):
        weekday_input = gr.Dropdown(
            choices=["星期一","星期二","星期三","星期四","星期五","星期六","星期日"],
            label="選擇星期",
            value="星期一"
        )
        btn_query = gr.Button("查詢課表")
        output_day = gr.Textbox(
            label="查詢結果",
            lines=15,
            interactive=False
        )

        btn_query.click(
            fn=query_by_weekday_gr,
            inputs=weekday_input,
            outputs=output_day
        )

    # Tab 2：今日上課提醒
    with gr.Tab("⏰ 今日上課提醒"):
        minutes_input = gr.Slider(
            minimum=10,
            maximum=180,
            step=10,
            value=60,
            label="未來幾分鐘內的課程？"
        )
        btn_upcoming = gr.Button("查詢即將開始的課程")
        output_upcoming = gr.Textbox(
            label="提醒結果",
            lines=15,
            interactive=False
        )

        btn_upcoming.click(
            fn=upcoming_classes_gr,
            inputs=minutes_input,
            outputs=output_upcoming
        )

    # Tab 3：AI 行前準備建議
    with gr.Tab("🤖 AI 行前準備"):
        weekday_ai = gr.Dropdown(
            choices=["星期一","星期二","星期三","星期四","星期五","星期六","星期日"],
            label="選擇要準備的星期",
            value="星期一"
        )
        btn_ai = gr.Button("產生行前重點")
        output_ai = gr.Textbox(
            label="AI 行前準備建議",
            lines=20,
            interactive=False
        )

        btn_ai.click(
            fn=ai_prep_tips,
            inputs=weekday_ai,
            outputs=output_ai
        )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ca6c6963a349e97228.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
